In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


In [2]:
# combine all df
df1 = pd.read_csv("result GREATEC.csv", parse_dates = True)
df2 = pd.read_csv("result UWC.csv", parse_dates = True)
df3 = pd.read_csv("result D&O.csv", parse_dates = True)
df4 = pd.read_csv("result GENETEC.csv", parse_dates = True)
df5 = pd.read_csv("result UNISEM.csv", parse_dates = True)
df6 = pd.read_csv("result FRONTKN.csv", parse_dates = True)
df7 = pd.read_csv("result ATAIMS.csv", parse_dates = True)

df9 = pd.read_csv("result FPI.csv", parse_dates = True)
df10 = pd.read_csv("result PENTA.csv", parse_dates = True)
df11 = pd.read_csv("result DUFU.csv", parse_dates = True)
df12 = pd.read_csv("result VS.csv", parse_dates = True)
df13 = pd.read_csv("result SKPRES.csv", parse_dates = True)
df14 = pd.read_csv("result MPI.csv", parse_dates = True)
df15 = pd.read_csv("result VITROX.csv", parse_dates = True)

# Merging all df
df_all = pd.concat([df1, df2, df3, df4, df5, df6, df7, df9, df10, df11, df12, df13, df14, df15], axis=0)


In [3]:
# sorting all values by dates
df_all = df_all.sort_values(by="Buy_Date")

In [4]:
# setting Dates as index
df_all.set_index('Buy_Date', inplace=True)
df_all

,Buy,Sell,Sell_Date,Pct_change,wr,Stock
Buy_Date,,,,,,
1994-09-07,4.58,4.46,1995-01-03,-2.620087,0,MPI
1994-09-27,4.72,4.46,1995-01-03,-5.508475,0,MPI
1994-09-30,4.80,4.46,1995-01-03,-7.083333,0,MPI
1994-09-30,2.64,2.25,1994-12-22,-14.772727,0,ATAIMS
1994-10-04,2.66,2.25,1994-12-22,-15.413534,0,ATAIMS
...,...,...,...,...,...,...
2021-01-11,6.22,7.40,2021-01-19,18.971061,1,UNISEM
2021-01-11,4.70,5.94,2021-01-20,26.382979,1,UWC
2021-01-11,5.01,5.70,2021-01-14,13.772455,1,PENTA


In [5]:
#Strategy Analysis

In [6]:
df_all["wr"].value_counts()

1    840
0    489
Name: wr, dtype: int64

In [7]:
total_winpct=round(840/(840+489)*100,3)

In [8]:
print(" Total win percentage is: {} %".format(total_winpct))

 Total win percentage is: 63.205 %


In [9]:

# SHARPE RATIO
str_return = df_all["Pct_change"].mean()
rf_rate = 0
volatility = df_all["Pct_change"].std()
print("Sharpe ratio of this strategy is: {}".format((str_return-rf_rate)/volatility))

Sharpe ratio of this strategy is: 0.23206536113048154


In [10]:
# Encoding the stock names
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_all["Stock"] = le.fit_transform(df_all["Stock"])
df_all

,Buy,Sell,Sell_Date,Pct_change,wr,Stock
Buy_Date,,,,,,
1994-09-07,4.58,4.46,1995-01-03,-2.620087,0,7
1994-09-27,4.72,4.46,1995-01-03,-5.508475,0,7
1994-09-30,4.80,4.46,1995-01-03,-7.083333,0,7
1994-09-30,2.64,2.25,1994-12-22,-14.772727,0,0
1994-10-04,2.66,2.25,1994-12-22,-15.413534,0,0
...,...,...,...,...,...,...
2021-01-11,6.22,7.40,2021-01-19,18.971061,1,10
2021-01-11,4.70,5.94,2021-01-20,26.382979,1,11
2021-01-11,5.01,5.70,2021-01-14,13.772455,1,8


### Resampling Techniques — Oversample minority class

In [11]:
from sklearn.utils import resample

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# Separate input features and target
X = df_all.drop(['wr','Pct_change','Sell_Date'], axis=1)
y = df_all.wr


In [14]:
df_all["wr"].value_counts()

1    840
0    489
Name: wr, dtype: int64

In [15]:
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.56, random_state=0, shuffle=False)

In [16]:
# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

In [17]:
X 

,Buy,Sell,Stock,wr
Buy_Date,,,,
1994-09-07,4.5800,4.4600,7,0
1994-09-27,4.7200,4.4600,7,0
1994-09-30,4.8000,4.4600,7,0
1994-09-30,2.6400,2.2500,0,0
1994-10-04,2.6600,2.2500,0,0
...,...,...,...,...
2010-08-12,0.0669,0.0525,8,0
2010-08-13,2.7500,2.8500,5,1
2010-08-16,0.0679,0.0525,8,0


In [18]:
# separate minority and majority classes
not_win = X[X.wr==0]
win = X[X.wr==1]

In [19]:
# upsample minority
not_win_upsampled = resample(not_win,
                          replace=True, # sample with replacement
                          n_samples=len(win), # match number in majority class
                          random_state=0) # reproducible results

In [20]:
# combine majority and upsampled minority
upsampled = pd.concat([win, not_win_upsampled]).sort_values(by="Buy_Date")

In [21]:
upsampled

,Buy,Sell,Stock,wr
Buy_Date,,,,
1994-09-07,4.5800,4.4600,7,0
1994-09-07,4.5800,4.4600,7,0
1994-09-27,4.7200,4.4600,7,0
1994-09-30,2.6400,2.2500,0,0
1994-09-30,2.6400,2.2500,0,0
...,...,...,...,...
2010-08-10,0.1050,0.1200,0,1
2010-08-12,0.0669,0.0525,8,0
2010-08-12,0.0669,0.0525,8,0


In [22]:
upsampled.wr.value_counts()

1    336
0    336
Name: wr, dtype: int64

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
# trying logistic regression again with the balanced dataset
y_train = upsampled.wr
X_train = upsampled.drop('wr', axis=1)

upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

In [25]:
# predicting the trainset
upsampled_pred_train = upsampled.predict(X_train)
trainscore = accuracy_score(y_train, upsampled_pred_train)
print("training score:", trainscore)

training score: 0.9032738095238095


In [26]:
# validating using test_set
upsampled_pred_test = upsampled.predict(X_test)
testscore = accuracy_score(y_test, upsampled_pred_test)
print("test score:", testscore)

test score: 0.9100671140939597


In [27]:
#check 
if trainscore - testscore < -0.01:
    print("Model Overfitted")
else:
    print("Model Ok")

Model Ok


In [28]:
trainscore - testscore

-0.006793304570150149

In [30]:
# classification report
from sklearn.metrics import classification_report
print("Training model score:\n {}\n".format(classification_report(y_train, upsampled_pred_train)))
print("Test model score:\n {}".format(classification_report(y_test, upsampled_pred_test)))

Training model score:
               precision    recall  f1-score   support

           0       0.88      0.93      0.91       336
           1       0.93      0.88      0.90       336

    accuracy                           0.90       672
   macro avg       0.90      0.90      0.90       672
weighted avg       0.90      0.90      0.90       672


Test model score:
               precision    recall  f1-score   support

           0       0.80      0.95      0.87       241
           1       0.98      0.89      0.93       504

    accuracy                           0.91       745
   macro avg       0.89      0.92      0.90       745
weighted avg       0.92      0.91      0.91       745

